# 07_experiments_cpu_svm_word_char_grid
# Experimentos: Modelo Robusto (TF-IDF word + char + Linear SVM)
# Objetivo:

Otimizar o desempenho do modelo TF-IDF (word + char) + Linear SVM por meio de uma busca eficiente de hiperparâmetros, identificando combinações que maximizem o F1-score e avaliando o potencial de ensemble entre os melhores modelos.

# O que ele faz:

1. Busca de hiperparâmetros em duas etapas para reduzir custo computacional e focar nas combinações mais promissoras.

Etapa A:

- Varredura inicial apenas do parâmetro C do LinearSVC (C ∈ {1.0, 2.0, 3.0, 4.0})
- Mantém fixos n-grams e max_features
- Objetivo: identificar rapidamente os valores de C mais promissores

Etapa B:

- Seleciona os dois melhores valores de C da Etapa A
- Testa combinações adicionais de hiperparâmetros, variando word n-grams (1,2) e (1,3) e char n-grams (3,5) e (3,6).
-  Objetivo: refinar a busca na região mais promissora

> Estratégia coarse-to-fine search para explorar o espaço de hiperparâmetros de forma eficiente.

2. Geração de tabela: config → F1 → threshold

3. Ensemble dos dois melhores modelos (média dos scores) com ajuste de threshold

# Observação:

Este notebook mantém a mesma estrutura do modelo base (06), incluindo:

- Pré-processamento (title + text, sem subject, remoção de duplicatas)
- Split estratificado
- Treino/validação com ajuste de threshold via decision_function

In [1]:
# 0) Importação das bibliotecas e carregamento dos dados

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.pipeline import FeatureUnion

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [2]:
## Pré-processamento (igual ao notebook base)

# content = title + text (sem subject)
train["content"] = train["title"].fillna("").astype(str) + " " + train["text"].fillna("").astype(str)
test["content"]  = test["title"].fillna("").astype(str)  + " " + test["text"].fillna("").astype(str)

# Remove duplicatas completas (title+text)
before = len(train)
train = train.drop_duplicates(subset=["title", "text"]).reset_index(drop=True)
print(f"Duplicatas removidas: {before - len(train)}")

X = train["content"].astype(str)
y = train["label"].astype(int)

print("Distribuição de classes (0=verdadeiro, 1=fake):")
print(y.value_counts())


Duplicatas removidas: 501
Distribuição de classes (0=verdadeiro, 1=fake):
label
0    16996
1     5347
Name: count, dtype: int64


In [3]:
# Split estratificado (igual ao notebook base)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)
print("Train:", X_train.shape[0], "Val:", X_val.shape[0])

Train: 17874 Val: 4469


In [4]:
## Funções auxiliares

# monta o pipeline mantendo a mesma ideia do notebook base
def build_pipeline(C=2.0, word_ngram=(1,2), char_ngram=(3,5), max_features=200000):
    # Word TF-IDF (conteúdo)
    word_tfidf = TfidfVectorizer(
        stop_words="english",
        ngram_range=word_ngram,
        min_df=2,
        max_features=max_features,
        sublinear_tf=True
    )

    # Char TF-IDF (estilo)
    char_tfidf = TfidfVectorizer(
        analyzer="char_wb",
        ngram_range=char_ngram,
        min_df=2,
        max_features=max_features,
        sublinear_tf=True
    )

    features = FeatureUnion([
        ("word", word_tfidf),
        ("char", char_tfidf)
    ])

    clf = LinearSVC(
        C=C,
        class_weight="balanced",
        random_state=42
    )

    pipeline = Pipeline([
        ("features", features),
        ("clf", clf)
    ])
    return pipeline


def best_threshold_for_f1(scores, y_true, n_grid=200):
    # Threshold grid entre percentis (evita extremos)
    lo = np.percentile(scores, 1)
    hi = np.percentile(scores, 99)
    thresholds = np.linspace(lo, hi, n_grid)

    best_t = 0.0
    best_f1 = -1.0
    for t in thresholds:
        pred_t = (scores >= t).astype(int)
        f1 = f1_score(y_true, pred_t)
        if f1 > best_f1:
            best_f1 = float(f1)
            best_t = float(t)
    return best_t, best_f1

# treina, calcula scores e encontra melhor threshold para F1
def fit_eval(pipeline, X_train, y_train, X_val, y_val):
    pipeline.fit(X_train, y_train)
    val_scores = pipeline.decision_function(X_val)
    best_t, best_f1 = best_threshold_for_f1(val_scores, y_val, n_grid=220)
    val_pred = (val_scores >= best_t).astype(int)
    return best_t, best_f1, val_scores, val_pred, pipeline


In [5]:
# Config base (igual ao notebook 06)

BASE_WORD = (1, 2)
BASE_CHAR = (3, 5)
BASE_MAXF = 200000

# Etapa A: testar C
Cs = [1.0, 2.0, 3.0, 4.0]

results = []
artifacts = {}  # guarda modelos e scores para possíveis ensembles

print("=== Etapa A: varredura de C (config base) ===")
for C in Cs:
    pipe = build_pipeline(C=C, word_ngram=BASE_WORD, char_ngram=BASE_CHAR, max_features=BASE_MAXF)
    best_t, best_f1, val_scores, val_pred, fitted = fit_eval(pipe, X_train, y_train, X_val, y_val)

    cfg = {"stage": "A", "C": C, "word": BASE_WORD, "char": BASE_CHAR, "max_features": BASE_MAXF}
    results.append({**cfg, "best_threshold": best_t, "f1_val": best_f1})

    # guardar artefatos
    key = f"A_C{C}_w{BASE_WORD}_c{BASE_CHAR}_mf{BASE_MAXF}"
    artifacts[key] = {"cfg": cfg, "pipeline": fitted, "val_scores": val_scores, "best_t": best_t, "best_f1": best_f1}

    print(f"C={C:.1f} | best_t={best_t:.4f} | F1={best_f1:.6f}")

# Seleciona top-2 C
dfA = pd.DataFrame(results).query("stage=='A'").sort_values("f1_val", ascending=False)
top2_C = dfA["C"].head(2).tolist()
print("\nTop-2 C:", top2_C)

# Etapa B: combinações para top-2 C
word_opts = [(1,2), (1,3)]
char_opts = [(3,5), (3,6)]

print("\n=== Etapa B: 2x2 combos de n-grams para top-2 C ===")
for C in top2_C:
    for w in word_opts:
        for ch in char_opts:
            pipe = build_pipeline(C=C, word_ngram=w, char_ngram=ch, max_features=BASE_MAXF)
            best_t, best_f1, val_scores, val_pred, fitted = fit_eval(pipe, X_train, y_train, X_val, y_val)

            cfg = {"stage": "B", "C": C, "word": w, "char": ch, "max_features": BASE_MAXF}
            results.append({**cfg, "best_threshold": best_t, "f1_val": best_f1})

            key = f"B_C{C}_w{w}_c{ch}_mf{BASE_MAXF}"
            artifacts[key] = {"cfg": cfg, "pipeline": fitted, "val_scores": val_scores, "best_t": best_t, "best_f1": best_f1}

            print(f"C={C:.1f} | word={w} | char={ch} | best_t={best_t:.4f} | F1={best_f1:.6f}")

# Tabela final de resultados
results_df = pd.DataFrame(results).sort_values(["f1_val","stage"], ascending=[False, True]).reset_index(drop=True)
print("\n=== Tabela (top 15) ===")
display(results_df.head(15))


=== Etapa A: varredura de C (config base) ===
C=1.0 | best_t=-0.1263 | F1=0.997662
C=2.0 | best_t=-0.0685 | F1=0.998129
C=3.0 | best_t=-0.0704 | F1=0.998129
C=4.0 | best_t=-0.0895 | F1=0.998129

Top-2 C: [2.0, 3.0]

=== Etapa B: 2x2 combos de n-grams para top-2 C ===
C=2.0 | word=(1, 2) | char=(3, 5) | best_t=-0.0685 | F1=0.998129
C=2.0 | word=(1, 2) | char=(3, 6) | best_t=-0.1625 | F1=0.997662
C=2.0 | word=(1, 3) | char=(3, 5) | best_t=-0.1246 | F1=0.997662
C=2.0 | word=(1, 3) | char=(3, 6) | best_t=-0.1492 | F1=0.997662
C=3.0 | word=(1, 2) | char=(3, 5) | best_t=-0.0704 | F1=0.998129
C=3.0 | word=(1, 2) | char=(3, 6) | best_t=-0.1646 | F1=0.997662
C=3.0 | word=(1, 3) | char=(3, 5) | best_t=-0.0779 | F1=0.998129
C=3.0 | word=(1, 3) | char=(3, 6) | best_t=-0.1554 | F1=0.997662

=== Tabela (top 15) ===


,stage,C,word,char,max_features,best_threshold,f1_val
0,A,2.0,"(1, 2)","(3, 5)",200000,-0.068503,0.998129
1,A,3.0,"(1, 2)","(3, 5)",200000,-0.070445,0.998129
2,A,4.0,"(1, 2)","(3, 5)",200000,-0.089508,0.998129
3,B,2.0,"(1, 2)","(3, 5)",200000,-0.068503,0.998129
4,B,3.0,"(1, 2)","(3, 5)",200000,-0.070445,0.998129
5,B,3.0,"(1, 3)","(3, 5)",200000,-0.077882,0.998129
6,A,1.0,"(1, 2)","(3, 5)",200000,-0.126303,0.997662
7,B,2.0,"(1, 2)","(3, 6)",200000,-0.162550,0.997662
8,B,2.0,"(1, 3)","(3, 5)",200000,-0.124572,0.997662
9,B,2.0,"(1, 3)","(3, 6)",200000,-0.149195,0.997662


In [6]:
# Pegando os 2 melhores da tabela completa (preferindo stage B se empatar)
best_two = results_df.sort_values(["f1_val","stage"], ascending=[False, True]).head(2)
print("=== Top 2 configs ===")
display(best_two)

# Recupera as chaves correspondentes em artifacts
def find_artifact_key(row):
    stage = row["stage"]
    C = row["C"]
    w = tuple(row["word"])
    ch = tuple(row["char"])
    mf = int(row["max_features"])
    # Monta o padrão de key como foi salvo
    if stage == "A":
        return f"A_C{C}_w{w}_c{ch}_mf{mf}"
    else:
        return f"B_C{C}_w{w}_c{ch}_mf{mf}"

k1 = find_artifact_key(best_two.iloc[0])
k2 = find_artifact_key(best_two.iloc[1])

a1 = artifacts[k1]
a2 = artifacts[k2]

scores_ens = (a1["val_scores"] + a2["val_scores"]) / 2.0
best_t_ens, best_f1_ens = best_threshold_for_f1(scores_ens, y_val.values, n_grid=300)
pred_ens = (scores_ens >= best_t_ens).astype(int)

print(f"\nEnsemble (média de scores) | best_t={best_t_ens:.4f} | F1={best_f1_ens:.6f}")
print("\nClassification report (val) — Ensemble:")
print(classification_report(y_val, pred_ens, digits=4))


=== Top 2 configs ===


,stage,C,word,char,max_features,best_threshold,f1_val
0,A,2.0,"(1, 2)","(3, 5)",200000,-0.068503,0.998129
1,A,3.0,"(1, 2)","(3, 5)",200000,-0.070445,0.998129



Ensemble (média de scores) | best_t=-0.0689 | F1=0.998129

Classification report (val) — Ensemble:
              precision    recall  f1-score   support

           0     0.9994    0.9994    0.9994      3400
           1     0.9981    0.9981    0.9981      1069

    accuracy                         0.9991      4469
   macro avg     0.9988    0.9988    0.9988      4469
weighted avg     0.9991    0.9991    0.9991      4469



In [7]:
# Monta pipelines com melhores configs
cfg1 = a1["cfg"]
cfg2 = a2["cfg"]

pipe1 = build_pipeline(C=cfg1["C"], word_ngram=tuple(cfg1["word"]), char_ngram=tuple(cfg1["char"]), max_features=int(cfg1["max_features"]))
pipe2 = build_pipeline(C=cfg2["C"], word_ngram=tuple(cfg2["word"]), char_ngram=tuple(cfg2["char"]), max_features=int(cfg2["max_features"]))

# Refit em todo o treino
pipe1.fit(X, y)
pipe2.fit(X, y)

# Scores no teste
test_X = test["content"].astype(str)
s1 = pipe1.decision_function(test_X)
s2 = pipe2.decision_function(test_X)
s_ens = (s1 + s2) / 2.0

test_pred_ens = (s_ens >= best_t_ens).astype(int)

submission = pd.DataFrame({
    "id": test["id"].values,
    "target": test_pred_ens
})

out_path = "submission_cpu_svm_word_char_ensemble.csv"
submission.to_csv(out_path, index=False)

print("Arquivo salvo:", out_path)
print(submission.head())
print("Linhas:", len(submission))


Arquivo salvo: submission_cpu_svm_word_char_ensemble.csv
      id  target
0   5398       1
1   5503       1
2  23151       0
3  12669       0
4  27864       0
Linhas: 5712


#Conclusão:

Este notebook apresentou a busca de hiperparâmetros para o modelo baseado em TF-IDF de palavras e caracteres combinado com Linear SVM, utilizando uma estratégia em duas etapas para explorar o espaço de configurações de forma eficiente. As melhores combinações obtiveram F1-score de 0.998129 no conjunto de validação.

Também foi avaliado um ensemble dos dois modelos com maior desempenho, porém o resultado obtido foi equivalente ao do melhor modelo individual (F1 = 0.998129). Assim, optou-se por utilizar o modelo individual como principal nas etapas posteriores de análise de erro e interpretabilidade, devido à maior simplicidade e facilidade de interpretação, sem prejuízo de desempenho.

Os resultados confirmam a alta eficácia da abordagem baseada em representações textuais híbridas (word + char n-grams) com classificadores lineares para a tarefa proposta.